## Lambda 층 사용하기

In [59]:
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# 나만의 함수를 정의합니다.
def custom_f(x):
    x_mean = K.mean(x)
    x_std = K.std(x)

    return (x - x_mean) / x_std

inputs = Input(shape = (5, ))
# Lambda 층을 통해 Layer으로 변환하여 연결합니다.
x = Lambda(custom_f)(inputs)
# 기존에 사용하던 층과 연결하여 사용할 수 있습니다.
# x = Dense(32, activation = 'relu')(x)

# 모델을 구성합니다.
model = Model(inputs, x)

# 간단한 예제
import tensorflow as tf

a = tf.Variable([[1., 2., 3., 4., 5.]])
a_mean = K.mean(a); a_std = K.std(a)
result_check = (a - a_mean) / a_std

print('모델의 출력값: ' + str(model(a)))
print('결과 확인: ' + str(result_check))

모델의 출력값: tf.Tensor([[-1.4142135  -0.70710677  0.          0.70710677  1.4142135 ]], shape=(1, 5), dtype=float32)
결과 확인: tf.Tensor([[-1.4142135  -0.70710677  0.          0.70710677  1.4142135 ]], shape=(1, 5), dtype=float32)


## 커스텀 케라스 층 사용하기

In [81]:
from tensorflow.keras.layers import Layer, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu

# 커스텀 층을 정의합니다.
class CustomLayer(Layer):
    def __init__(self, num_hidden):
        super(CustomLayer, self).__init__()
        self.num_hidden = num_hidden
    
    # 가중치를 정의합니다.
    def build(self, input_shape):
        self.kernels = self.add_weight('kernels',
                                       shape = [int(input_shape[-1]), self.num_hidden])
        
        self.bias = self.add_weight('bias',
                                   shape = [self.num_hidden])
    
    # 수행할 연산을 정의합니다.
    def call(self, x):
        return relu(tf.matmul(x, self.kernels) + self.bias)
    
    # 출력값의 형태를 명시해줍니다.
    def compute_output_shape(self, input_shape):
        return [input_shape[0], self.num_hidden]

# 모델을 구성합니다.
inputs = Input(shape = (5, ))
x = CustomLayer(32)(inputs)
model = Model(inputs, x)

## 커스텀 함수와 Dense 층 파라미터 수 비교

In [82]:
model.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
custom_layer_10 (CustomLayer (None, 32)                192       
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________


In [83]:
inputs = Input(shape = (5, ))
x = Dense(32)(inputs)

model = Model(inputs, x)
model.summary()

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                192       
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________
